In [66]:
from bs4 import BeautifulSoup
import codecs
import re
import pandas as pd
from os import listdir
from os.path import isfile, join

In [67]:
mypath = "www.pherobase.com/database/compound/"
compoundfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
compoundfiles = ["www.pherobase.com/database/compound/" + f for f in compoundfiles if "compounds-detail-" in f]

## From the side of infochemicals

In [68]:
listaformula, listaCAS, listaMW, listaFeromona, listaespecies, listaP, listaA, listaAl, listaK, listaSy, listaNones, listauses, listaCs= ([] for i in range(13))

def getdatafromfile(file):
    fdata = codecs.open(file, 'r', encoding='utf-8', errors='ignore')
    soup = BeautifulSoup(fdata)
    
    formula = [td.find_next_sibling("td").text for td in soup.find_all("td",text="Formula:")][0]
    CAS = [td.find_next_sibling("td").text for td in soup.find_all("td",text="CAS#:")][0]
    MW = float([td.find_next_sibling("td").text for td in soup.find_all("td",text="MW:")][0])
    
    td = soup.find_all(text=re.compile('.*{0}.*'.format("in its chemical communication system")))[0]
    my_table = td.parent.parent.parent
    
    tipoFeromona = []
    rows = my_table.findChildren(['th', 'tr'], recursive=True)
    for row in rows:
        cells = row.findChildren('font', {"color":False})
        for cell in cells:
            value = cell.string
            #if value!= None and value!="*":
            if value!="*":
                tipoFeromona.append(value)
    
    especies = []
    especies.append("")
    rows = my_table.findChildren(['th', 'tr'], recursive=True)
    for row in rows:
        cells = row.findChildren('a')
        for cell in cells:
            value = cell.string
            if value != None and especies[-1]=="":
                especies[-1] = value
            elif value != None and especies[-1]!="":
                especies[-1] = especies[-1] + " " + value
            elif value==None and especies[-1]!="":
                especies.append("")
    especies = especies[0:-1]
    return(formula, CAS, MW, tipoFeromona, especies)

In [69]:
def get_number_of_C(formula):
    totalC = 0
    flag = False
    for i, s in enumerate(formula): 
        if s =="C":
            flag = True
            t=i+1
            number = ""
            while formula[t].isdigit():
                number = number +formula[t]
                t+=1
            if number=="":
                totalC0=1
            else:
                totalC += int(number)
    if flag==False:
        print(formula)
    return(totalC)


for i, file in enumerate(compoundfiles):
    if i%100==0: print("Quedan: "  +str(len(compoundfiles)-i))
    formula, CAS, MW, tipoFeromona, especies = getdatafromfile(file)
    if len(tipoFeromona)!=len(especies): 
        print("CAUTION ERROR "+ file)
        break
    NumberCarbons = get_number_of_C(formula)
    if NumberCarbons >0:
        listaformula.append(formula)
        listaCs.append(NumberCarbons)
        listaCAS.append(CAS)
        listaMW.append(MW)
        listaFeromona.append(tipoFeromona)
        listaespecies.append(especies)
        listaP.append(tipoFeromona.count("P"))
        listaA.append(tipoFeromona.count("A"))
        listaAl.append(tipoFeromona.count("Al"))
        listaK.append(tipoFeromona.count("K"))
        listaSy.append(tipoFeromona.count("Sy"))
        listaNones.append(tipoFeromona.count(None))
        listauses.append(len(especies))


Quedan: 1400
Quedan: 1300
Quedan: 1200
Quedan: 1100
Quedan: 1000
Quedan: 900
Quedan: 800
Quedan: 700
Quedan: 600
HNO2
Quedan: 500
Quedan: 400
Quedan: 300
Quedan: 200
H3N
Quedan: 100


In [65]:
len(listaformula)

3196

In [70]:
dfChemicals = pd.DataFrame({'formula': listaformula, 'Cs': listaCs, 'CAS': listaCAS, "MW":listaMW, "feromonas": listaFeromona, 
                            "especies": listaespecies, "totaluses":listauses, "P": listaP, "A": listaA, 
                            "Al": listaAl, "K": listaK, "Sy":listaSy})

dfChemicals.to_csv('chemicals.csv', index=False)

In [75]:
dfChemicals[dfChemicals.Cs!=0]

,formula,Cs,CAS,MW,feromonas,especies,totaluses,P,A,Al,K,Sy
0,C22H44O2,22,36528-28-6,340.58,"[P, P]","[Centris analis, Pogonomyrmex barbatus]",2,2,0,0,0,0
1,C6H12O2,6,868-57-5,116.16,"[A, P, P, Al, Al, P, Al, Al, P, P, P, Al, P, A...","[Cyclocephala literata, Graphium doson albidum...",19,10,1,8,0,0
2,C19H36O2,19,,296.49,"[P, P, P, P, P, P, P]","[Gephyromantis boulengeri, Mantella aurantiaca...",7,7,0,0,0,0
3,C33H64,33,,460.86,"[P, P, P]","[Drosophila arizonae, Drosophila mojavensis, D...",3,3,0,0,0,0
4,C10H16N2,10,40790-29-2,164.25,"[P, P, P, P, P, P, P, P, P, P, P]","[Dinoroseobacter shibae, Loktanella sp, Tachyt...",11,11,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1388,C7H12O4,7,,160.17,"[P, P, P, P, P, P, P]","[Godyris kedema, Hymenitis andromica, Hymeniti...",7,7,0,0,0,0
1389,C14H26O2,14,5684-69-5,226.36,"[P, P, P, P, P, P, P, P, P, Al, P, Al, P]","[Bactrothrips carbonarius, Bactrothrips flecto...",13,11,0,2,0,0
1390,C8H18O,8,111-87-5,130.23,"[P, Al, P, A, A, A, A, Al, Al, Al, P, P, P, P,...","[Armadillidium sp, Varroa jacobsoni, Polycentr...",44,25,13,6,0,0
1391,C7H14O,7,,114.19,[P],[Smicridea annulicornis],1,1,0,0,0,0
